In [ ]:
import  pandas as pd
import  os
from    datetime import datetime
from    tqdm import tqdm
import  regex as re
import  time
import  numpy as np
import  random
import  pyperclip as py
# import  PyPDF2
import  openai 
import  regex as re
import  random 
import  os


from    dotenv import load_dotenv
load_dotenv()
openai.api_type     = os.getenv('OPENAI_TYPE')
openai.api_base     = os.getenv('OPENAI_BASE')
openai.api_version  = os.getenv('OPENAI_VERSION')
openai.api_key      = os.getenv('OPENAI_KEY')


In [2]:
openai.api_key, openai.api_base, openai.api_version, openai.api_type

('e12bcf7f25f94e259d06c393b4568bc0',
 'https://dcei-openai-we.openai.azure.com/',
 '2023-03-15-preview',
 'azure')

In [3]:
recipes         = pd.read_csv('recipes_2.csv')
ingredients_df  = pd.read_csv('ingredients_df.csv')
pantry          = pd.read_csv('matching.csv')
# lets go to the download folder and import the ingredients_3.csv file
conversion      = pd.read_csv('conversion.csv')
gpt_recipes     = pd.read_csv('gpt_recipes.csv')
fridge          = pd.DataFrame(columns=['ingredients', 'amounts'])

In [4]:
# 
def calculate_cost_df(df):
    """
    Calculate the total cost of purchasing ingredients for a dish.

    Args:
    df (pd.DataFrame): DataFrame containing dish ingredients and prices.

    Returns:
    pd.DataFrame: DataFrame with an additional column for the total cost of purchasing ingredients for each dish.
    """
    # Calculate the cost for purchasing each ingredient
    # Assuming 'price' is the price for 'purchase_size' amount of the ingredient
    for item in df['ingredients'].unique():
        price = df[df['ingredients'] == item]['price'].values[0]
        amount = df[df['ingredients'] == item]['amounts'].values[0]
        purchase_size = df[df['ingredients'] == item]['purchase_size'].values[0]

        if amount > purchase_size:
            upper = np.ceil(amount / purchase_size)
            purchase_cost = upper * price
        else:
            purchase_cost = price

        df.loc[df['ingredients'] == item, 'price'] = purchase_cost
            
    # Calculate the total cost for each dish
    total_cost = df.groupby('urls')['price'].sum().reset_index()

    # Merge total cost back into the original dataframe
    df = df.merge(total_cost, on='urls', how='left', suffixes=('', '_total'))

    return df

def ingredient_divider(url):
    local_ingredient_amount = len(pantry[pantry.urls == url])
    local_matches = len(pantry[(pantry.urls == url) & (pantry.in_recipe == True)])
    if local_ingredient_amount == 0 or local_matches == 0:
        return 0
    else:
        return local_matches / local_ingredient_amount

def matching_ingredients(url):
    sample_ingredients = pantry[pantry.urls == url].ingredients.tolist()
    pantry['in_recipe'] = False
    pantry.loc[pantry.ingredients.isin(sample_ingredients), 'in_recipe'] = True
    recipes['matches'] = recipes.urls.apply(ingredient_divider)

def matching_ingredient_df(df):
    sample_ingredients = df.ingredients.tolist()
    pantry['in_recipe'] = False
    pantry.loc[pantry.ingredients.isin(sample_ingredients), 'in_recipe'] = True
    recipes['matches'] = recipes.urls.apply(ingredient_divider)

def check_fridge_availability(ingredient, amount_needed):
    """
    Check if the ingredient is available in the fridge and if there is enough of it.

    Args:
    ingredient (str): The name of the ingredient.
    amount_needed (float): The amount of the ingredient needed.

    Returns:
    float: The amount that still needs to be purchased.
    """
    if ingredient in fridge['ingredients'].values:
        available_amount = fridge[fridge['ingredients'] == ingredient]['amounts'].values[0]
        if available_amount >= amount_needed and amount_needed > 0:
            # Enough available, no need to purchase more
            return 0
        else:
            # Some amount still needs to be purchased
            return amount_needed - available_amount
    else:
        # Ingredient not in fridge, need to purchase the entire amount
        return amount_needed

def calculate_cost_and_remaining(row):
    amount_to_purchase = check_fridge_availability(row['ingredients'], row['amounts'])
    if row['purchase_size'] == 0 or pd.isna(row['purchase_size']):
        # Handle the case where purchase_size is zero or NaN
        # You can return a default value or handle it as per your requirement
        return pd.Series([0, 0], index=['ingredient_cost', 'remaining'])
    
    if amount_to_purchase > 0:
        if amount_to_purchase > row['purchase_size']:
            num_units = np.ceil(amount_to_purchase / row['purchase_size'])
            cost = num_units * row['local_price']
            remaining = (num_units * row['purchase_size']) - amount_to_purchase
        else:
            cost = row['local_price']
            remaining = row['purchase_size'] - amount_to_purchase
    else:
        cost = 0
        remaining = 0  # Or adjust based on what's left in the fridge
    
    return pd.Series([cost, remaining], index=['ingredient_cost', 'remaining'])

# Existing function
def calculate_cost_of_urls(urls):
    # Assuming combined_df is your DataFrame containing the grocery list
    combined_df = pantry[pantry.urls.isin(urls)].copy()
    # Group by ingredients to sum the amounts needed for each
    grouped_df = combined_df.groupby('ingredients').agg({'amounts': 'sum', 'purchase_size': 'first', 'local_price': 'first'}).reset_index()

    # Apply the function to calculate the cost and remaining amount for each ingredient
    grouped_df[['ingredient_cost', 'remaining']] = grouped_df.apply(calculate_cost_and_remaining, axis=1)

    # Summing up the total cost
    total_cost = grouped_df['ingredient_cost'].sum()

    # print(f"Total Cost: {total_cost}")
    grouped_df['cumsum'] = grouped_df['ingredient_cost'].cumsum()
    return grouped_df



In [5]:
ingredients_df

,ingredients,pareto,purchase_unit,type,price,purchase_size,count
0,smør,True,g,0.0,26.0,200.0,577
1,olivenolie,True,ml,0.0,160.0,1000.0,570
2,sukker,True,g,0.0,25.0,1000.0,566
3,hvidløg,True,fed,1.0,30.0,12.0,502
4,hvedemel,True,g,0.0,18.0,2000.0,458
...,...,...,...,...,...,...,...
803,julesalat,False,NaN,NaN,NaN,NaN,1
804,kageplast,False,NaN,NaN,NaN,NaN,1
805,dryp balsamico glace,False,NaN,NaN,NaN,NaN,1
806,dryp honning,False,NaN,NaN,NaN,NaN,1


In [6]:
gpt_recipes

,urls,ingredients,amounts,unit
0,recipe 0,grøntsagsbouillon,194.00,NaN
1,recipe 0,hvidløg,9.00,fed
2,recipe 0,løg,5.00,stk
3,recipe 0,olivenolie,999.00,ml
4,recipe 0,parmesan,125.00,g
...,...,...,...,...
6322,recipe 436,rød chili,0.25,stk
6323,recipe 436,rød peberfrugt,2.50,stk
6324,recipe 436,safran,0.50,stk
6325,recipe 436,squash,0.50,stk


In [9]:
url = gpt_recipes.urls.sample(1).values[0]
local_df = gpt_recipes[gpt_recipes.urls == url].copy()


In [10]:
def call_model(text):
    messages = [
        {'role': 'system', 'content':
            """
            Du er en erfaren kok - der er pedantisk og præcis
            Du har en stor kærlighed til mad og du elsker at lave mad
            Du er til en konkurrence i at lave vegetarisk mad, som er sundt og velsmagende.
            Fokus skal være på at lave original mad, som er som de vil lave det lokalt og i proffessionel kontekst, men som tager under 45 minutter.
            
            Til konkurencen skal du lave en ret til to personer, som er inspireret som ikke har noget med to andre retter, og som kun bruger de ingredienser du er tildelt. 
            Du kan hvis det er nødvendigt adgang til pasta, ris og kartoffler. 
            
            Dommerne til konkurrencen er meget glade for følgende retter:
                - Tacos
                - One-pot pasta
            
            Dommerne er meget opmærksomme på spild, så det er vigtigt, at du ikke bruger for meget af nogle råvarerne.
                - Pasta, Ris, Kartofler -> 250 Gram
                - Oliven Olie -> 15 ml
            
            
            Du skal leverer en opskrift i nedenstående format:
            Navn: 
            [rettens navn]
            Ingredienser:
            [antal] [enhed] [ingrediens]
            [antal] [enhed] [ingrediens]
            Fremgangsmåde:
            [fremgangsmåde]
            
            """},
        {'role': 'user', 'content':
            f"""
            Lav en ret udfra nedenstående ingredienser, du kan tilføje 250 gram pasta, ris eller kartofler hvis det er nødvendigt. 
            
            Dommerne til konkurrencen er meget glade for følgende retter:
                - Tacos
                - Klassiske italianske retter
                - Klassiske franske retter
                - One-pot pasta
            
            Dommerne er meget opmærksomme på spild, så det er vigtigt, at du ikke bruger for meget af nogle råvarerne.
                - Pasta, Ris, Kartofler -> 250 Gram
                - Oliven Olie -> 15 ml
                - hvidløg -> 2  fed
            
            Ingredienser:
            {text}
            """}
    ]
    
    response = openai.ChatCompletion.create(
        model = 'gpt-35-turbo',
        messages = messages,
        max_tokens = 500,
        temperature = 0.2,
        engine = 'gpt-35-turbo'
    )

    return response['choices'][0]['message']['content']

# # now lets get the top 5 recipes, which are dinners
# top_5 = recipes[(recipes.dinner == True) & (recipes.vegeterian == True)].sort_values('matches', ascending=False).head(5)
# top_5 = top_5[top_5.matches == top_5.matches.min()]
# suggestion = top_5.text.values[0].split('-')[0]
# suggestion

# data = call_model(y)

In [19]:
measurements = conversion.purchase_unit.unique().tolist()

def ingredients_cleaner(text):
    try:
        text = text.split('Ingredienser')[1]
        text = text.split('Fremgangsmåde')[0]
        text = text.strip()
    except:
        text = ''
    return text
         
def url_generator(text):
    if text.startswith('Navn:'):
        return text.splitlines()[0].split('Navn:')[1].strip()
    else:
        random_number = random.randint(0, 100)
        return f'gpt_recipe_{random_number}'

def ingredients_setter(ingredients_text):
    url = url_generator(ingredients_text)
    
    ingredients_text = ingredients_cleaner(ingredients_text)    

    for line in ingredients_text.splitlines():
        if '- ' in line:
            line = line.split('- ')[1]
    # now lets see if the first item is a number or not
        pattern = '\d+'
    # lets utelise the starts with function here, and combine it with some regex
        match = re.findall(pattern, line)
        local_measurement = ''
        if len(match) > 0:
            
            # if we get a match, we know that the first item is a number
            # so lets split the line at the first space
            amount = line.split(' ', 1)[0]
            # and then print the first item
            # now lets see if there is any measurements in the line
            for measurement in measurements:
                if f' {measurement} ' in line:
                # if there is, lets split the line at the measurement
                    local_measurement = measurement
                # and then print the first item
                    break
            else:
            # if there is no measurement, just print the first item
                local_measurement = 'stk'

            if ',' in line:
                new_line = line.split(', ')[0]
            else:
                new_line = line

            if '.' in new_line:
                new_line = line.split('. ')[0]
            else:
                new_line = new_line

            new_line = new_line.replace(amount, '')
            new_line = new_line.replace(f' {local_measurement} ', '')
            new_line = new_line.strip()
        
            if ',' in amount:
                amount = amount.replace(',', '.')
                                        # url, amount, unit               ingre     pur, s, p, ps, lp, p, ir, m
            pantry.loc[len(pantry)] = ([url, amount, local_measurement, new_line, '', 0, 0, 0, 0, 0, 0, 0])


In [20]:
pantry

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.00,spsk,mayonnaise,g,0.0,40.0,350.0,40.0,True,True,0.173913
1,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.00,stk,æg,stk,0.0,36.0,10.0,36.0,True,True,0.173913
2,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.50,tsk,røget paprika,stk,1.0,0.0,1.0,0.0,True,False,0.173913
3,https://www.valdemarsro.dk/indisk-vegetar-bowl/,250.00,g,blomkål,stk,0.0,30.0,1.0,30.0,True,True,0.173913
4,https://www.valdemarsro.dk/indisk-vegetar-bowl/,50.00,g,pankorasp,NaN,NaN,NaN,NaN,0.0,False,False,0.173913
...,...,...,...,...,...,...,...,...,...,...,...,...
17033,http://www.valdemarsro.dk/lemoncurd/,0.75,dl,citronsaft,NaN,NaN,NaN,NaN,0.0,False,False,0.400000
17034,http://www.valdemarsro.dk/lemoncurd/,1.50,stk,æg,stk,0.0,36.0,10.0,36.0,True,True,0.400000
17035,http://www.valdemarsro.dk/lemoncurd/,100.00,g,sukker,g,0.0,25.0,1000.0,25.0,True,False,0.400000
17036,http://www.valdemarsro.dk/lemoncurd/,0.25,stk,øko citron,NaN,NaN,NaN,NaN,0.0,True,False,0.400000


In [21]:
gpt_recipes[gpt_recipes.urls == url]

,urls,ingredients,amounts,unit
0,recipe 0,grøntsagsbouillon,194.0,NaN
1,recipe 0,hvidløg,9.0,fed
2,recipe 0,løg,5.0,stk
3,recipe 0,olivenolie,999.0,ml
4,recipe 0,parmesan,125.0,g
5,recipe 0,piskefløde,26.0,ml
6,recipe 0,rasp,485.0,g
7,recipe 0,soltørrede tomater i olie,70.0,g
8,recipe 0,spinat,200.0,g
9,recipe 0,squash,0.5,stk


In [22]:

def texitify_df(my_df):
    y = ''
    my_df = my_df.reset_index(drop=True)
    for i in range(len(my_df)):
        row = my_df.iloc[i]
        y += str(row.amounts) + ' ' + str(row.unit) + ' ' + str(row.ingredients) + '\n'
    return y

In [28]:
pantry =    pd.read_csv('pantry.csv')
recipes =   pd.read_csv('recipes_2.csv')

In [29]:
from tqdm import tqdm

errors = 0

with tqdm(total=len(gpt_recipes.urls.unique())) as pbar:
    for url in gpt_recipes.urls.unique():
        try:
            local_df        = gpt_recipes[gpt_recipes.urls == url].copy()
            input_text      = texitify_df(local_df)
            response        = call_model(input_text)
            name            = url_generator(response)
            ingredients_setter(response)
            recipes.loc[len(recipes)] = [name, response, 0, 0, 0, 0, 0, 0]
            # break
        except:
            errors += 1
            pass
        pbar.update(1)
        pbar.set_description(f'Processing: {url}, Errors: {errors}, I just made: {name}')



Processing: recipe 436, Errors: 3, I just made: Kikærte og avocado salat med persille og tomat: 100%|██████████| 437/437 [42:53<00:00,  5.89s/it]                          


In [30]:
recipes.to_csv('recipes_3.csv', index=False)
pantry.to_csv('pantry_2.csv', index=False)

In [31]:
pantry

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,spsk,mayonnaise,g,0.0,40.0,350.0,40.0,True,0,0.173913
1,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,stk,æg,stk,0.0,36.0,10.0,36.0,True,1,0.173913
2,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.5,tsk,røget paprika,stk,1.0,0.0,1.0,0.0,True,0,0.173913
3,https://www.valdemarsro.dk/indisk-vegetar-bowl/,250.0,g,blomkål,stk,0.0,30.0,1.0,30.0,True,0,0.173913
4,https://www.valdemarsro.dk/indisk-vegetar-bowl/,50.0,g,pankorasp,NaN,NaN,NaN,NaN,0.0,False,0,0.173913
...,...,...,...,...,...,...,...,...,...,...,...,...
27052,Kikærte og avocado salat med persille og tomat,50,g,bredbladet persille,,0.0,0.0,0.0,0.0,0,0,0.000000
27053,Kikærte og avocado salat med persille og tomat,1/2,stk,rød chili,,0.0,0.0,0.0,0.0,0,0,0.000000
27054,Kikærte og avocado salat med persille og tomat,2,fed,hvidløg,,0.0,0.0,0.0,0.0,0,0,0.000000
27055,Kikærte og avocado salat med persille og tomat,1/2,stk,citron,,0.0,0.0,0.0,0.0,0,0,0.000000


In [32]:
recipes

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,Indisk vegetarbowl med rispilaf og sprød blomk...,0.0,1,0,1,7714.0,0.434783
1,https://www.valdemarsro.dk/pasta-med-graeskar/,Pasta med græskar og masser af god smag - opsk...,0.0,1,0,1,6415.0,0.363636
2,https://www.valdemarsro.dk/blomkaal-og-halloum...,Blomkål og halloumi i sursød sauce med dejlig ...,0.0,1,0,1,6278.0,0.250000
3,https://www.valdemarsro.dk/graeskarmuffins/,Græskarmuffins - opskrift på saftige muffins m...,0.0,0,0,0,5100.0,0.200000
4,https://www.valdemarsro.dk/vegetar-taco/,Vegetar taco med krydret bagt blomkål og mango...,0.0,1,0,1,4967.0,0.250000
...,...,...,...,...,...,...,...,...
3255,Kikærte og mango curry med basmati ris,Navn: Kikærte og mango curry med basmati ris\n...,0.0,0,0,0,0.0,0.000000
3256,Kikærte- og kyllingefad med avocado og cherryt...,Navn: Kikærte- og kyllingefad med avocado og c...,0.0,0,0,0,0.0,0.000000
3257,Persille-ris med kikærter og yoghurt-dressing,Navn: Persille-ris med kikærter og yoghurt-dre...,0.0,0,0,0,0.0,0.000000
3258,Avocado og kikærte tacos,Navn: Avocado og kikærte tacos\n\nIngredienser...,0.0,0,0,0,0.0,0.000000


In [ ]:
input_text = texitify_df(local_df)
input_text

'0.5 stk aubergine\n100.0 g champignon\n0.5 stk frisk basilikum\n199.0 nan grøntsagsbouillon\n5.5 stk gulerod\n0.5 stk hakkede tomater\n9.5 fed hvidløg\n998.0 g kartoffel\n115.0 g koncentreret tomatpuré\n4.5 stk løg\n998.5 ml olivenolie\n0.5 stk porre\n499.0 g rasp\n0.5 stk rosmarin\n0.5 stk røget paprika\n105.0 g soltørrede tomater i olie\n0.5 stk squash\n999.5 ml sødmælk\n0.5 stk timian\n'

In [ ]:
response = call_model(input_text)
response

'Navn: One-Pot Pasta med Grøntsager og Soltørrede Tomater\n\nIngredienser:\n- 250 g pasta\n- 15 ml olivenolie\n- 2 fed hvidløg, finthakket\n- 5,5 gulerødder, skåret i små tern\n- 0,5 aubergine, skåret i små tern\n- 0,5 squash, skåret i små tern\n- 100 g champignon, skåret i skiver\n- 0,5 porre, skåret i skiver\n- 0,5 dåse hakkede tomater\n- 199 ml grøntsagsbouillon\n- 115 g koncentreret tomatpuré\n- 105 g soltørrede tomater i olie, hakket\n- 0,5 tsk røget paprika\n- 0,5 tsk rosmarin\n- 0,5 tsk timian\n- 0,5 dl frisk basilikum, hakket\n- 499 g rasp\n- 999,5 ml sødmælk\n- Salt og peber efter smag\n\nFremgangsmåde:\n1. Varm olivenolie i en stor gryde ved medium varme. Tilsæt hvidløg og steg i et minut, indtil det er duftende.\n2. Tilsæt gulerødder, aubergine, squash, champignon og porre. Steg i 5 minutter, indtil grøntsagerne er bløde.\n3. Tilsæt hakkede tomater, grøntsagsbouillon, koncentreret tomatpuré, soltørrede tomater, røget paprika, rosmarin, timian og basilikum. Rør godt rundt og 

In [ ]:
print(response)

Navn: One-Pot Pasta med Grøntsager og Soltørrede Tomater

Ingredienser:
- 250 g pasta
- 15 ml olivenolie
- 2 fed hvidløg, finthakket
- 5,5 gulerødder, skåret i små tern
- 0,5 aubergine, skåret i små tern
- 0,5 squash, skåret i små tern
- 100 g champignon, skåret i skiver
- 0,5 porre, skåret i skiver
- 0,5 dåse hakkede tomater
- 199 ml grøntsagsbouillon
- 115 g koncentreret tomatpuré
- 105 g soltørrede tomater i olie, hakket
- 0,5 tsk røget paprika
- 0,5 tsk rosmarin
- 0,5 tsk timian
- 0,5 dl frisk basilikum, hakket
- 499 g rasp
- 999,5 ml sødmælk
- Salt og peber efter smag

Fremgangsmåde:
1. Varm olivenolie i en stor gryde ved medium varme. Tilsæt hvidløg og steg i et minut, indtil det er duftende.
2. Tilsæt gulerødder, aubergine, squash, champignon og porre. Steg i 5 minutter, indtil grøntsagerne er bløde.
3. Tilsæt hakkede tomater, grøntsagsbouillon, koncentreret tomatpuré, soltørrede tomater, røget paprika, rosmarin, timian og basilikum. Rør godt rundt og bring det i kog.
4. Tilsæt 

In [ ]:
recipes = pd.read_csv('recipes_2.csv')

remove  = pantry.urls.value_counts()[:3].reset_index()['index'].to_list()

pantry  = pantry[~pantry.urls.isin(remove)]

pantry

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,spsk,mayonnaise,g,0.0,40.0,350.0,40.0,True,0,0.173913
1,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,stk,æg,stk,0.0,36.0,10.0,36.0,True,1,0.173913
2,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.5,tsk,røget paprika,stk,1.0,0.0,1.0,0.0,True,0,0.173913
3,https://www.valdemarsro.dk/indisk-vegetar-bowl/,250.0,g,blomkål,stk,0.0,30.0,1.0,30.0,True,0,0.173913
4,https://www.valdemarsro.dk/indisk-vegetar-bowl/,50.0,g,pankorasp,NaN,NaN,NaN,NaN,0.0,False,0,0.173913
...,...,...,...,...,...,...,...,...,...,...,...,...
20481,Spinat og squash pasta med soltørrede tomater,70,g,soltørrede tomater i olie,,0.0,0.0,0.0,0.0,0,0,0.000000
20482,Spinat og squash pasta med soltørrede tomater,194,g,grøntsagsbouillon,,0.0,0.0,0.0,0.0,0,0,0.000000
20483,Spinat og squash pasta med soltørrede tomater,200,g,spinat,,0.0,0.0,0.0,0.0,0,0,0.000000
20484,Spinat og squash pasta med soltørrede tomater,26,ml,piskefløde,,0.0,0.0,0.0,0.0,0,0,0.000000


In [ ]:
# pantry = pantry[pantry.index < 16991].copy()

In [ ]:
recipes.to_csv('recipes_3.csv', index=False)
pantry.to_csv('pantry.csv', index=False)

In [ ]:
pantry

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,spsk,mayonnaise,g,0.0,40.0,350.0,40.0,True,0,0.173913
1,https://www.valdemarsro.dk/indisk-vegetar-bowl/,1.0,stk,æg,stk,0.0,36.0,10.0,36.0,True,1,0.173913
2,https://www.valdemarsro.dk/indisk-vegetar-bowl/,0.5,tsk,røget paprika,stk,1.0,0.0,1.0,0.0,True,0,0.173913
3,https://www.valdemarsro.dk/indisk-vegetar-bowl/,250.0,g,blomkål,stk,0.0,30.0,1.0,30.0,True,0,0.173913
4,https://www.valdemarsro.dk/indisk-vegetar-bowl/,50.0,g,pankorasp,NaN,NaN,NaN,NaN,0.0,False,0,0.173913
...,...,...,...,...,...,...,...,...,...,...,...,...
21498,One-Pot Pasta med Spinat og Soltørrede Tomater,26.0,ml,piskefløde,,0.0,0.0,0.0,0.0,0,0,0.000000
21499,One-Pot Pasta med Spinat og Soltørrede Tomater,250.0,g,pasta,,0.0,0.0,0.0,0.0,0,0,0.000000
21500,One-Pot Pasta med Spinat og Soltørrede Tomater,70.0,g,soltørrede tomater i olie,,0.0,0.0,0.0,0.0,0,0,0.000000
21501,One-Pot Pasta med Spinat og Soltørrede Tomater,200.0,g,spinat,,0.0,0.0,0.0,0.0,0,0,0.000000


In [ ]:
recipes

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
0,https://www.valdemarsro.dk/indisk-vegetar-bowl/,Indisk vegetarbowl med rispilaf og sprød blomk...,0.0,True,False,True,7714.0,0.217391
1,https://www.valdemarsro.dk/pasta-med-graeskar/,Pasta med græskar og masser af god smag - opsk...,0.0,True,False,True,6415.0,0.363636
2,https://www.valdemarsro.dk/blomkaal-og-halloum...,Blomkål og halloumi i sursød sauce med dejlig ...,0.0,True,False,True,6278.0,0.250000
3,https://www.valdemarsro.dk/graeskarmuffins/,Græskarmuffins - opskrift på saftige muffins m...,0.0,False,False,False,5100.0,0.266667
4,https://www.valdemarsro.dk/vegetar-taco/,Vegetar taco med krydret bagt blomkål og mango...,0.0,True,False,True,4967.0,0.062500
...,...,...,...,...,...,...,...,...
3258,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.000000
3259,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.000000
3260,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.000000
3261,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.000000


In [ ]:
recipes[recipes.urls == 'One-Pot Pasta med Spinat og Soltørrede Tomater']

,urls,text,fullfillment,dinner,addon,vegeterian,price_add,matches
2826,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
2827,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
2828,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
2829,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
2830,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
...,...,...,...,...,...,...,...,...
3258,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
3259,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
3260,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0
3261,One-Pot Pasta med Spinat og Soltørrede Tomater,Navn: One-Pot Pasta med Spinat og Soltørrede T...,0.0,True,False,True,0.0,0.0


In [ ]:
pantry[pantry.urls == 'One-Pot Pasta med Spinat og Soltørrede Tomater']

,urls,amounts,units,ingredients,purchase_unit,spice,price,purchase_size,local_price,pareto,in_recipe,matches
17038,One-Pot Pasta med Spinat og Soltørrede Tomater,194.0,stk,nan grøntsagsbouillon,,0.0,0.0,0.0,0.0,0,0,0.0
17039,One-Pot Pasta med Spinat og Soltørrede Tomater,2,fed,hvidløg,,0.0,0.0,0.0,0.0,0,0,0.0
17040,One-Pot Pasta med Spinat og Soltørrede Tomater,1,stk,løg,,0.0,0.0,0.0,0.0,0,0,0.0
17041,One-Pot Pasta med Spinat og Soltørrede Tomater,15,ml,olivenolie,,0.0,0.0,0.0,0.0,0,0,0.0
17042,One-Pot Pasta med Spinat og Soltørrede Tomater,125.0,g,parmesan,,0.0,0.0,0.0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21498,One-Pot Pasta med Spinat og Soltørrede Tomater,26.0,ml,piskefløde,,0.0,0.0,0.0,0.0,0,0,0.0
21499,One-Pot Pasta med Spinat og Soltørrede Tomater,250.0,g,pasta,,0.0,0.0,0.0,0.0,0,0,0.0
21500,One-Pot Pasta med Spinat og Soltørrede Tomater,70.0,g,soltørrede tomater i olie,,0.0,0.0,0.0,0.0,0,0,0.0
21501,One-Pot Pasta med Spinat og Soltørrede Tomater,200.0,g,spinat,,0.0,0.0,0.0,0.0,0,0,0.0


In [ ]:
measurements = conversion.purchase_unit.unique().tolist()